# Integration of da/dt

In [ ]:
import numpy as np
import holodeck as holo
from holodeck.constants import YR, GYR
from holodeck import utils, cosmo

SHAPE = 20

sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)

# hard = holo.hardening.Hard_GW
hard = holo.hardening.Fixed_Time.from_sam(sam, 2 * GYR)

